<a href="https://colab.research.google.com/github/raphaellins/ai-investiments/blob/master/mini_indice_5_2020_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime

In [2]:
df = pd.read_csv('/content/drive/MyDrive/MLAZ/win_5_2020.csv', usecols=['Date', 'Open', 'High', 'Low', 'Close', 'Tick', 'Volume'])

In [ ]:
df.head()

In [ ]:
df.dtypes

1. Analyzing if the candle closed is High or Low and adding a column with this information

In [4]:
df['state'] = df.Open < df.Close

In [ ]:
df.head()

2. Adding the info about the candle body size and shadow low and shadow high (Percentage). The entire candle is 100% including the shadows. 

In [9]:
# candle size = High - Low


df['candle_size'] = df.High - df.Low
df['body_size'] = abs(df.Open - df.Close)
df['shade_low'] = np.where(df['state'] == True, (df.Open - df.Low), (df.Close - df.Low))
df['shade_high'] = np.where(df['state'] == True, (df.High - df.Close), (df.High - df.Open))


In [10]:
df.head()

,Date,Open,High,Low,Close,Tick,Volume,state,body_size,shade_low,shade_high,candle_size
0,2020.01.02 09:00,118940,119042,118889,119016,12658,61538,True,76,51,26,153
1,2020.01.02 09:05,119011,119113,118915,119098,16174,72643,True,87,96,15,198
2,2020.01.02 09:10,119098,119123,119016,119021,12359,49335,False,77,5,25,107
3,2020.01.02 09:15,119021,119042,118981,118996,9840,39651,False,25,15,21,61
4,2020.01.02 09:20,119001,119016,118965,119001,7253,32205,False,0,36,15,51


In [ ]:
df_limit = df.iloc[0:50]

fig = go.Figure(data=[go.Candlestick(x=df_limit['Date'],
                open=df_limit['Open'],
                high=df_limit['High'],
                low=df_limit['Low'],
                close=df_limit['Close'])])

fig.show()

3. Let`s identify the candle with some Price Action Patterns

In [15]:
df['doji'] = df.body_size < df.candle_size * 0.03
df['hammer'] = np.where(df.shade_low > df.body_size * 2 & df.shade_high < df.body_size * 0.1, True, False) 
df['inverted_hammer'] = np.where(df.shade_low < df.body_size*0.1 & df.shade_high > df.body_size * 2, True, False)  

ValueError: ignored

In [12]:
df.head()

,Date,Open,High,Low,Close,Tick,Volume,state,body_size,shade_low,shade_high,candle_size,doji
0,2020.01.02 09:00,118940,119042,118889,119016,12658,61538,True,76,51,26,153,False
1,2020.01.02 09:05,119011,119113,118915,119098,16174,72643,True,87,96,15,198,False
2,2020.01.02 09:10,119098,119123,119016,119021,12359,49335,False,77,5,25,107,False
3,2020.01.02 09:15,119021,119042,118981,118996,9840,39651,False,25,15,21,61,False
4,2020.01.02 09:20,119001,119016,118965,119001,7253,32205,False,0,36,15,51,True
